Run: docker compose up -d

In [2]:
import numpy as np

from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct
from sentence_transformers import SentenceTransformer


In [3]:
METRIC_TYPE = "COSINE" # Która metoda ma być użyta do porównywania
TEXT_TO_SEARCH = 30         # Numer sentencji do porównania
NN_NUMBER = 5               # Ilość sąsiadów do odszukania
COLLECTION_NAME = "sentences_collection"

# Lista metod porównujących
METRIC_LIST = {
    "COSINE": Distance.COSINE,
    "EUCLID": Distance.EUCLID,
    "DOT": Distance.DOT
}

In [4]:
# Dane tekstowe
sentences = [
    "Słońce świeci na niebie.", # 0
    "Dzisiaj jest piękna pogoda.",
    "Lubię uczyć się programowania.",
    "Python to popularny język programowania.",
    "Wieczorem często czytam książki.",
    "Mój pies lubi biegać po parku.",
    "Programowanie to świetna zabawa.",
    "Niebo jest dziś bardzo niebieskie.",
    "Czytanie rozwija wyobraźnię.",
    "Chodzę codziennie na spacer z psem.",
    "Lubię Mazury, morze i góry.", # 10
    "Java to mój ulubiony język programowania.",
    "Ostatnio częściej programuję w Pythonie",
    "SQL też nie jest zły.",
    "Za tydzień jedziemy całą ekipą na wakacje w góry.",
    "Zawsze na wyjazdy wakacyjne zabieramy naszego psa.",
    "Dostałem wczoraj mandat za przektoczenie predkości.",
    "Jazda moim Porshe wyzwala we mnie mega emocje.",
    "Kolarstwo górskie daje mi wiele frajdy.",
    "Wczoraj byłem w kinie.",
    "Mam uczulenie na czekoladę ale za to mogę jeść marmoladę.", # 20
    "Najlepsze pączki są z marmoladą i lukrem.",
    "Twój tort urodzinowy był pyszny.",
    "Od roku ograniczam cukier i czuję sie wyśmienicie.",
    "Wczoraj kupiłem poradnik pt. Dieta dla każdego.",
    "Sposób odżywiania ma wpływ na nasze samopoczucie.",
    "Właśnie skończyłem czytać fajną książkę.",
    "Czytanie książek to moja ulubiona zabawa.",
    "Wczoraj byłem w bibliotek i wypożyczyłem kilka książek na temat programowania.",
    "Wczoraj wróciliśmy z wakacji z Mazur.",
    "Mój pies lubi gonić koty." # 30
]

In [5]:
# Tworzenie embeddingów
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(sentences).astype(np.float32)

# Inicjalizacja klienta Qdrant (lokalnie)
client = QdrantClient(host="qdrant_nosql_lab", port=6333)

# Nazwa kolekcji
collection_name = COLLECTION_NAME

# Tworzenie kolekcji z metryką cosine
client.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=embeddings.shape[1], distance=METRIC_LIST[METRIC_TYPE])
)

# Dodanie punktów (embedding + tekst jako payload)
points = [
    PointStruct(id=i, vector=embeddings[i], payload={"text": sentences[i]})
    for i in range(len(sentences))
]

client.upsert(collection_name=collection_name, points=points)

# Zapytanie
query_vector = embeddings[TEXT_TO_SEARCH]

print(f"\n🔎 Zapytanie: {sentences[TEXT_TO_SEARCH]}")

# Wyszukiwanie 3 najbliższych
results = client.search(
    collection_name=collection_name,
    query_vector=query_vector,
    limit=NN_NUMBER,
    with_payload=True
)

# Wyniki
print("\n🔗 Najbardziej podobne zdania:")
for hit in results:
    text = hit.payload["text"]
    score = hit.score
    print(f"- \"{text}\" (similarity: {score:.4f})")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/tmp/ipykernel_537/2411939690.py:12: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(



🔎 Zapytanie: Mój pies lubi gonić koty.

🔗 Najbardziej podobne zdania:
- "Mój pies lubi gonić koty." (similarity: 1.0000)
- "Mój pies lubi biegać po parku." (similarity: 0.7829)
- "Dzisiaj jest piękna pogoda." (similarity: 0.5105)
- "Czytanie książek to moja ulubiona zabawa." (similarity: 0.4536)
- "Kolarstwo górskie daje mi wiele frajdy." (similarity: 0.4354)


/tmp/ipykernel_537/2411939690.py:31: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(


In [ ]:
# client.delete_collection(collection_name=COLLECTION_NAME)

True

In [10]:
collections = client.get_collections()
print(collections)

collections=[]
